In [1]:
import numpy as np
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import gc
from IPython.display import display, clear_output

Using TensorFlow backend.


In [2]:
POS_TO_LETTER = 'abcdefghjklmnop'
LETTER_TO_POS = {letter: pos for pos, letter in enumerate(POS_TO_LETTER)}

IMG_SIZE = 15

def to_move(pos):
    return POS_TO_LETTER[pos[1]] + str(pos[0] + 1)

def to_pos(move):
    return int(move[1:]) - 1, LETTER_TO_POS[move[0]]

def get_train(file_name):
    file = open("/home/dessous/renju/data/" + file_name)
    states = []
    labels = []
    num = 1
    for line in tqdm(file.readlines()):
        res, *moves = line.split()
        field = np.zeros((15, 15), dtype='int')
        for move in moves:
            states.append(np.copy(field))
            cur_move = to_pos(move)
            labels.append(cur_move[0] * 15 + cur_move[1])
            field[cur_move[0], cur_move[1]] = num
            num = -num
    return np.array(states), np.array(labels)

In [3]:
def generate_model():
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))


    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    '''
    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    '''
    model.add(Flatten())
    model.add(Dense(225))
    model.add(Activation('softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [4]:
model = load_model('/home/dessous/renju/model_v1.h5')

In [ ]:
model = generate_model()
model.summary()

In [ ]:
BATCH_SIZE = 50
EPOCHS_COUNT = 3

for i in range(48):
    train, labels = get_train("train" + str(i))
    train = np.reshape(train, (train.shape[0], train.shape[1], train.shape[2], 1))
    model.fit(train, labels, batch_size=BATCH_SIZE, epochs=EPOCHS_COUNT, verbose=1)
    del train
    del labels
    gc.collect()


In [21]:
class Player(enum.IntEnum):
    NONE = 0
    BLACK = -1
    WHITE = 1

    def another(self):
        return Player(-self)

    def __repr__(self):
        if self == Player.BLACK:
            return 'black'
        elif self == Player.WHITE:
            return 'white'
        else:
            return 'none'

    def __str__(self):
        return self.__repr__()

class Game:
    width, height = 15, 15
    shape = (width, height)
    line_length = 5
    max_game_length = 30

    def __init__(self):
        self._result = Player.NONE
        self._player = Player.BLACK
        self._board = numpy.full(self.shape, Player.NONE, dtype=numpy.int8)
        self._positions = list()

    def __bool__(self):
        return self.result() == Player.NONE and \
            len(self._positions) < max_game_length

    def move_n(self):
        return len(self._positions)

    def player(self):
        return self._player

    def result(self):
        return self._result

    def board(self):
        return self._board

    def positions(self, player=Player.NONE):
        if not player:
            return self._positions

        begin = 0 if player == Player.BLACK else 1
        return self._positions[begin::2]

    def dumps(self):
        return ' '.join(map(util.to_move, self._positions))

    @staticmethod
    def loads(dump):
        game = Game()
        for pos in map(util.to_pos, dump.split()):
            game.move(pos)
        return game


    def is_posible_move(self, pos):
        return 0 <= pos[0] < self.height \
            and 0 <= pos[1] < self.width \
            and not self._board[pos]

    def move(self, pos):
        assert self.is_posible_move(pos), 'impossible pos: {pos}'.format(pos=pos)

        self._positions.append(pos)
        self._board[pos] = self._player

        if not self._result and util.check(self._board, pos):
            self._result = self._player
            return

        self._player = self._player.another()
        
    def print_field(self):
        board = """
           a   b   c   d   e   f   g   h   j   k   l   m   n   o   p
         =============================================================
       1 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
       2 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
       3 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
       4 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
       5 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
       6 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
       7 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
       8 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
       9 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
      10 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
      11 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
      12 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
      13 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
      14 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         |---+---+---+---+---+---+---+---+---+---+---+---+---+---+---|
      15 | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c | %c |
         =============================================================
        """
        to_print = self.field.copy()
        to_print *= 79
        to_print[to_print == -79] = 88
        to_print[to_print == 0] = 45
        print(board % tuple(to_print.flatten()))
        


In [22]:
def predict_turn(model, field):
    probs = model.predict(field.reshape((1, IMG_SIZE, IMG_SIZE, 1)))
    while True:
        num = np.argmax(probs)
        if field[num // 15, num % 15]:
            probs[0, num] -= 1
        else:
            break
    return np.argmax(probs)

def pos(x, y):
    return x * IMG_SIZE + y

In [ ]:
game = Game()
while True:
    clear_output()
    num = predict_turn(model, game.field)
    game.turn(num)
    game.print_field()
    move = input()
    x, y = to_pos(move)
    game.turn(pos(x, y))
    gc.collect()

In [ ]:
game.print_field()